# Black Jack milestone project
- The Udemy Black Jack milestone project
## Overview 
- The aim of the card game is to get as close to 21 without going over.
- Initially the player and dealer are dealt two cards each. THe deal has one card face down and one face up.
- The player makes a bet on how likely they think they are to win the round. 
- The player can then hit or hold. Hit gives the player another card. Hold means keep the current hand. 
- The dealer then takes his turn. 
- A dealers 21 beats a players 21.
- If the dealer wins, the player looses their bet.
- If the player wins, they double their money. 
- The game continues until the player has lost their money or they decided to stop.
- An Ace can be 1 or 11.

In [1]:
import random
from time import sleep
from IPython.display import clear_output

#Global Variables
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

In [21]:
class card():
    
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        self.value = values[rank]
    
    def __str__(self):
        return self.rank+" of "+self.suit
    
class deck():
    
    def __init__(self):
        self.all_cards = []
        
        #Generate Deck on instantiation of a deck
        for suit in suits:
            for rank in ranks:
                self.all_cards.append(card(rank,suit))
    
    def __str__(self):
        for card in self.all_cards:
            print(card)
            
    def shuffle(self):
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        return self.all_cards.pop()

class gamer():
    
    def __init__(self,name):
        self.name = name
        self.hand = []
        self.hand_value = 0
    
    def show_hand(self):
        print(f"{self.name}'s hand: ")
        
        for card in self.hand:
            print(f" {card}")
        
        print(f" Value: {self.hand_value}")
        
    def calc_hand_value(self):
        
        No_of_Aces = 0
        self.hand_value = 0
        
        #Sum hand and check for aces
        for card in self.hand:
            self.hand_value += card.value 
            if card.rank == 'Ace':
                No_of_Aces += 1
        
        #Hand value calulations for varying number of aces
        if No_of_Aces == 0:
            pass
            
        elif No_of_Aces == 1:
            if self.hand_value > 21:
                self.hand_value -= 10
            else:
                pass
        
        elif No_of_Aces > 1:
            self.hand_value -= 10*(No_of_Aces-1)
            if self.hand_value > 21:
                self.hand_value -= 10

class player(gamer):
    
    def __init__(self,name):
        gamer.__init__(self,name)
        self.bank_balance = 100
        self.bet = 0
        
    def make_bet(self):
        
        print(f"\nBank Balance: {self.bank_balance}")
        
        while self.bet > self.bank_balance or self.bet == 0:
            try:
                self.bet = int(input('Place a bet: '))
            except:
                print('Please put in a number')
            else:
                if self.bet <= 0:
                    print('Please put in a number greater than 0')
                    continue

                elif self.bet > self.bank_balance:
                    print("You can't bet more than you own!")
                    continue
        
        self.bank_balance -= self.bet
    
    def decide(self):
        
        choice = None
        
        while True:
            
            try:
                choice = input("Hit or Hold? ").capitalize()
            except:
                pass
            
            if choice not in ["Hit", "Hold"]:
                print("Please only select Hit or Hold")

            else:
                break
                    
        
        return choice
    
class dealer(gamer):
    
    def __init__(self,name='Dealer'):
        gamer.__init__(self,name)
        self.hand = []
        self.hand_value = 0
        self.name = name
    
    def decide(self):
        if self.hand_value < 16:
            return 'Hit'
        else:
            return 'Hold'

def deal(player, dealer, deck):
    for x in range(2):
        player.hand.append(deck.deal_one())
        dealer.hand.append(deck.deal_one())
        
def show_table(player, dealer):
    clear_output(wait=True)
    player.show_hand()
    print('\n')
    print("Dealer's Hand: ")
    print(f" {dealer.hand[0]}")
    print(' Hidden Card')

def return_cards(player, dealer, deck):
    for x in range(len(player.hand)):
        deck.all_cards.append(player.hand.pop())
    
    for x in range(len(dealer.hand)):
        deck.all_cards.append(dealer.hand.pop())

def take_go(player, deck):
    
    clear_output(wait=True)
    print(f"{player.name}'s Go!")
    sleep(1.5)
    clear_output(wait=True)
    player.show_hand()
    
    while True:
        
        choice = player.decide()
        
        if choice == 'Hold':
            return choice

        elif choice == 'Hit':
            clear_output(wait=True)
            player.hand.append(deck.deal_one())
            player.calc_hand_value()
            player.show_hand()

        if player.hand_value > 21:
            print("\nBUST!")
            return 'Bust'

def finish_routine(player, dealer, deck):
    return_cards(player, dealer, deck)
    player.bet = 0
    print(f"\nBank_Balance: {player.bank_balance}")   

In [23]:
#Setup
player1 = player(input("Insert Name: "))
Dealer = dealer()
Deck = deck()

#Play while there are still funds
while True:
    
    #Deal a hand
    Deck.shuffle()
    deal(player1, Dealer, Deck)
    
    #Show hand
    player1.calc_hand_value()
    Dealer.calc_hand_value()
    show_table(player1, Dealer)
    
    #Place a bet
    player1.make_bet()

    #Player1 Turn
    while True:

        #Player 1s go
        player1_outcome = take_go(player1, Deck)

        #If player one goes bust
        if player1_outcome == 'Bust':
            print(f"\n{player1.name} BUST")
            break

        #If player decides to hold
        elif player1_outcome == 'Hold':

            #Dealer Turn
            Dealer_outcome = take_go(Dealer, Deck)

            #If dealer goes bust
            if Dealer.hand_value > 21:
                print('\nDealer BUST!')
                player1.bank_balance += player1.bet*2
                break

            #If dealer decides to hold
            elif Dealer.hand_value <= 21:

                #Check who has won
                if player1.hand_value > Dealer.hand_value:
                    print(f"\n{player1.name} is the winner!")
                    player1.bank_balance += player1.bet*2
                    break

                elif player1.hand_value <= Dealer.hand_value:
                    print("\nDealer wins")
                    break

    #Return cards, display bank balance and check if player wants another round.
    finish_routine(player1, Dealer, Deck)
    
    #End Game conditions
    keep_playing = None
    
    if player1.bank_balance == 0:
        print('The money has all run out!\n END!')
        break
        
    else:
        
        while keep_playing not in ['Yes', 'No']:
            
            keep_playing = input("Keep Playing?").capitalize()
            
            if keep_playing not in ['Yes','No']:
                print('Not a valid answer')
        if keep_playing == 'Yes':
            continue
            
        else:
            break


Ben's hand: 
 Eight of Clubs
 Seven of Hearts
 Jack of Spades
 Value: 25

BUST!

Ben BUST

Bank_Balance: 0
The money has all run out!
 END!
